In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 400)

In [2]:
df1 = pd.read_csv('9479-proc.csv')
df2 = pd.read_csv('9471-proc.csv')
df3 = pd.read_csv('9466-proc.csv')
df4 = pd.read_csv('9458-proc.csv')
df5 = pd.read_csv('9457-proc.csv')
df6 = pd.read_csv('3748-proc.csv')
facts = pd.read_csv('all_cases_facts.csv')

In [3]:
gold_standard = df1.append([df2, df3, df4, df5, df6], ignore_index = True)

In [4]:
facts = pd.read_csv('all_cases_facts.csv')
pd.options.display.max_rows = 100

def edit_distance(s1, s2):
    m=len(s1)+1
    n=len(s2)+1

    tbl = {}
    for i in range(m): tbl[i,0]=i
    for j in range(n): tbl[0,j]=j
    for i in range(1, m):
        for j in range(1, n):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            tbl[i,j] = min(tbl[i, j-1]+1, tbl[i-1, j]+1, tbl[i-1, j-1]+cost)

    return tbl[i,j]

list(gold_standard)
facts

,ID,conclusion,docket_number,facts_of_the_case,name,question,term
0,62327,"<p>In a 4-to-1 decision, the justices held tha...",None,"<p>In 1777, the Executive Council of Georgia a...",Chisholm v. Georgia,<p>Was the state of Georgia subject to the jur...,1789-1850
1,62328,<p>The Court held that the tax was legitimate....,None,"<p>In 1794, Congress enacted a law entitled ""A...",Hylton v. United States,<p>Did Congress violate the Constitution and g...,1789-1850
2,62329,<p>Four of the five justices wrote opinions. I...,None,"<p>This case involved the Treaty of Paris, whi...",Ware v. Hylton,<p>Does the Treaty of Paris override an otherw...,1789-1850
3,62330,"<p>In a unanimous decision, the Court held tha...",None,"<p>Mr. and Mrs. Caleb Bull, the stated benefic...",Calder v. Bull,<p>Was the Connecticut legislation a violation...,1789-1850
4,62331,"<p>Yes, yes, no. Chief Justice John Marshall d...",None,<p>In the last days of President John Adams’ p...,Marbury v. Madison,<p>(1) Do the plaintiffs have a right to recei...,1789-1850
5,62332,"<p>No. In a unanimous decision, the Court affi...",None,<p>The USS <em>Boston</em> captured the Danish...,Little v. Barreme,<p>Did the President have the authority to iss...,1789-1850
6,62333,"<p>In a unanimous opinion, the Court held that...",None,"<p>In 1795, the Georgia state legislature pass...",Fletcher v. Peck,<p>Could the contract between Fletcher and Pec...,1789-1850
7,62334,<p>Yes. The unanimous Court held that the tax-...,None,"<p>In 1758, the New Jersey colonial legislatur...",New Jersey v. Wilson,<p>Did the repeal of the tax exemption impair ...,1789-1850
8,62335,<p>The Court rejected the claim that Virginia ...,None,<p>Lord Fairfax held land in Virginia. He was ...,Martin v. Hunter's Lessee,<p>Does the appellate power of the Supreme Cou...,1789-1850
9,62336,"<p>In a unanimous decision, the Court held tha...",None,"<p>In 1816, Congress chartered The Second Bank...",McCulloch v. Maryland,<ol><li>Did Congress have the authority to est...,1789-1850


In [5]:
def check_term(fact,against):
    if '-' in fact:
        ff = fact.split("-")
        if int(against) >= int(ff[0]) and int(against) <= int(ff[1]):
            return True
    elif int(fact) == int(against):
        return True
    return False

def lookup(df,s,s2,term):
    dec_set = {}
    for i,r in df.iterrows():
        if r['caseName'] and s.lower() in str(r['caseName']).lower() and s2.lower() in str(r['caseName']).lower() and check_term(term,r['term']):
            if r['US_CITE'] in dec_set:
                dec_set[r['US_CITE']] +=1
            else:
                dec_set[r['US_CITE']] = 1
    return dec_set

In [6]:
FACTS_CASE_NAME = facts['name']
GOLD_STANDARD_CASE_NAME = gold_standard['caseName']

In [7]:
gold_standard_nn = gold_standard[gold_standard['caseName'].notnull()]

In [ ]:
matches = []
for i,f in facts.iterrows():
    dec_setter = {}
    print ("working on",i,"---------------------------------------",f["name"],"-------",f['ID'])
    cn = f['name']
    cn = cn.lower()
    cn = cn.split(" ")
    for i in range(len(cn)-1):
        for j in cn[i+1:]:
            if len(cn[i]) > 4 and len(j) > 4 and cn[i] not in ['united','states','southern'] and j not in ['united','states','southern']:
                dc = lookup(gold_standard_nn,cn[i],j,f['term'])
                for k,v in dc.items():
                    if k in dec_setter:
                        dec_setter[k] += dc[k]
                    else:
                        dec_setter[k] = dc[k]
    l_max = 0
    l_max_key = None
    for k,v in dec_setter.items():
        if v>l_max:
            l_max_key = k
            l_max = v
    
    if l_max_key:
        matches.append(k)
    else:
        matches.append(None)

working on 0 --------------------------------------- Chisholm v. Georgia ------- 62327
working on 1 --------------------------------------- Hylton v. United States ------- 62328
working on 2 --------------------------------------- Ware v. Hylton ------- 62329
working on 3 --------------------------------------- Calder v. Bull ------- 62330
working on 4 --------------------------------------- Marbury v. Madison ------- 62331
working on 5 --------------------------------------- Little v. Barreme ------- 62332
working on 6 --------------------------------------- Fletcher v. Peck ------- 62333
working on 7 --------------------------------------- New Jersey v. Wilson ------- 62334
working on 8 --------------------------------------- Martin v. Hunter's Lessee ------- 62335
working on 9 --------------------------------------- McCulloch v. Maryland ------- 62336
working on 10 --------------------------------------- Trustees of Dartmouth College v. Woodward ------- 62337
working on 11 ---------

working on 87 --------------------------------------- Coyle v. Smith ------- 62413
working on 88 --------------------------------------- Weeks v. United States ------- 62414
working on 89 --------------------------------------- The Los Angeles Switching Case ------- 62415
working on 90 --------------------------------------- Shreveport Rate Cases ------- 62416
working on 91 --------------------------------------- Coppage v. Kansas ------- 62417
working on 92 --------------------------------------- Bunting v. Oregon ------- 62418
working on 93 --------------------------------------- Buchanan v. Warley ------- 62419
working on 94 --------------------------------------- Hammer v. Dagenhart ------- 62420
working on 95 --------------------------------------- Frohwerk v. United States ------- 62421
working on 96 --------------------------------------- Debs v. United States ------- 62422
working on 97 --------------------------------------- Schenck v. United States ------- 62423
working on 98

working on 170 --------------------------------------- Ex parte Quirin ------- 62492
working on 171 --------------------------------------- Wickard v. Filburn ------- 62493
working on 172 --------------------------------------- Parker v. Brown ------- 62494
working on 173 --------------------------------------- Murdock v. Pennsylvania ------- 62495
working on 174 --------------------------------------- Martin v. City of Struthers ------- 62496
working on 175 --------------------------------------- West Virginia State Board of Education v. Barnette ------- 62497
working on 176 --------------------------------------- Hirabayashi v. United States ------- 62498
working on 177 --------------------------------------- Federal Power Commission v. Hope Natural Gas Company ------- 62499
working on 178 --------------------------------------- Yakus v. United States ------- 62500
working on 179 --------------------------------------- Smith v. Allwright ------- 62501
working on 180 -----------------

working on 251 --------------------------------------- Chessman v. Teets ------- 59196
working on 252 --------------------------------------- Corn Products Refining Company v. Commissioner of Internal Revenue ------- 59197
working on 253 --------------------------------------- Hyun v. Landon ------- 59198
working on 254 --------------------------------------- United States v. Twin City Power Company ------- 59199
working on 255 --------------------------------------- United States v. Zucca ------- 59200
working on 256 --------------------------------------- Steiner v. Mitchell ------- 59201
working on 257 --------------------------------------- United Mine Workers of America v. Arkansas Oak Floorings Company ------- 59202
working on 258 --------------------------------------- Slochower v. Board of Higher Education of New York City ------- 59203
working on 259 --------------------------------------- Hatahley v. United States ------- 59204
working on 260 ---------------------------------

working on 326 --------------------------------------- Cold Metal Process Company v. United Engineering & Foundry Company ------- 59274
working on 327 --------------------------------------- National Labor Relations Board v. Coca-Cola Bottling Company of Louisville, Inc. ------- 59275
working on 328 --------------------------------------- Indian Towing Company, Inc. v. United States ------- 59276
working on 329 --------------------------------------- International Harvester Credit Corporation v. Goodrich ------- 59277
working on 330 --------------------------------------- Black v. Cutter Laboratories ------- 59278
working on 331 --------------------------------------- United States v. Storer Broadcasting Company ------- 59279
working on 332 --------------------------------------- Griffin v. Illinois ------- 59280
working on 333 --------------------------------------- Sugden et ux. v. United States ------- 62271
working on 334 --------------------------------------- Texas v. New Mexico 

...

working on 7563 --------------------------------------- Maracich v. Spears ------- 55969
working on 7564 --------------------------------------- Adoptive Couple v. Baby Girl ------- 55966
working on 7565 --------------------------------------- Walden v. Fiore ------- 55985
working on 7566 --------------------------------------- Medtronic, Inc. v. Boston Scientific Corp. ------- 56000
working on 7567 --------------------------------------- King v. Burwell ------- 56136
working on 7568 --------------------------------------- Prado Navarette v. California ------- 55983
working on 7569 --------------------------------------- DaimlerChrysler AG v. Bauman ------- 55980
working on 7570 --------------------------------------- Lexmark International v. Static Control Components ------- 55994
working on 7571 --------------------------------------- PPL Corporation v. Commissioner of Internal Revenue ------- 55958
working on 7572 --------------------------------------- Marshall v. Rodgers ------- 6

working on 7644 --------------------------------------- Alice Corporation v. CLS Bank International ------- 62260
working on 7645 --------------------------------------- Alabama Department of Revenue v. CSX Transportation, Inc. ------- 56098
working on 7646 --------------------------------------- Argentina v. NML Capital, Ltd. ------- 62256
working on 7647 --------------------------------------- United States v. Kwai Fun Wong ------- 56053
working on 7648 --------------------------------------- Zivotofsky v. Kerry ------- 56102
working on 7649 --------------------------------------- City of Los Angeles v. Patel ------- 56058
working on 7650 --------------------------------------- Dart Cherokee Basin Operating Company LLC v. Owens ------- 56111
working on 7651 --------------------------------------- Wellness International Network v. Sharif ------- 56126
working on 7652 --------------------------------------- Oneok, Inc. et al. v. Learjet, Inc. et al. ------- 56082
working on 7653 ------

working on 7722 --------------------------------------- Equal Employment Opportunity Commission v. Abercrombie & Fitch Stores, Inc. ------- 56154
working on 7723 --------------------------------------- Sturgeon v. Frost ------- 62589
working on 7724 --------------------------------------- Kingsley v. Hendrickson ------- 56150
working on 7725 --------------------------------------- Montgomery v. Louisiana ------- 56177
working on 7726 --------------------------------------- Michigan v. Environmental Protection Agency ------- 56148
working on 7727 --------------------------------------- Carroll v. Carman ------- 56144
working on 7728 --------------------------------------- Puerto Rico v. Sanchez Valle ------- 62598
working on 7729 --------------------------------------- Bruce v. Samuels ------- 62307
working on 7730 --------------------------------------- Campbell-Ewald Company v. Gomez ------- 62167
working on 7731 --------------------------------------- Bullard v. Blue Hills Bank, fka 

working on 7805 --------------------------------------- Gloucester County School Board v. G.G. ------- 62765
working on 7806 --------------------------------------- Sessions v. Morales-Santana ------- 62738
working on 7807 --------------------------------------- Cooper v. Harris ------- 62723
working on 7808 --------------------------------------- Pena-Rodriguez v. Colorado ------- 62694
working on 7809 --------------------------------------- Utah v. Strieff ------- 62585
working on 7810 --------------------------------------- Samsung Electronics Co. v. Apple Inc. ------- 62690
working on 7811 --------------------------------------- Midland Funding v. Johnson ------- 62770
working on 7812 --------------------------------------- Caetano v. Massachusetts ------- 62691
working on 7813 --------------------------------------- Turner v. United States ------- 62786
working on 7814 --------------------------------------- Manuel v. City of Joliet ------- 62650
working on 7815 ------------------

working on 7886 --------------------------------------- PEM Entities LLC v. Levin ------- 62870
working on 7887 --------------------------------------- Hall v. Hall ------- 62893
working on 7888 --------------------------------------- Byrd v. United States ------- 62887
working on 7889 --------------------------------------- Dunn v. Madison ------- 62919
working on 7890 --------------------------------------- Trump v. International Refugee Assistance Project ------- 62918
working on 7891 --------------------------------------- Dalmazzi v. United States ------- 62891
working on 7892 --------------------------------------- City of Hays, Kansas v. Vogt ------- 62888
working on 7893 --------------------------------------- Minnesota Voters Alliance v. Mansky ------- 62923
working on 7894 --------------------------------------- Encino Motorcars, LLC v. Navarro ------- 62892
working on 7895 --------------------------------------- Texas v. New Mexico and Colorado ------- 62911
working on 7896 

In [ ]:
facts['MATCH_ID'] = matches
facts.to_csv('FINAL_MATCH_FACTS_FILE.csv')